In [111]:
import os
from datetime import datetime, timedelta

import git
import pandas as pd
import plotly.express as px

# here it's good to get our bearings

In [112]:
os.getcwd()

'/Users/timottinger/Projects/gitminer-jupyter/OldNotebooks'

# read a repository to graph

In [113]:
def create_tag_interval_df(repo: git.Repo, since: datetime) -> pd.DataFrame:
    source = ((tag_ref.name, tag_ref.commit.authored_datetime) for tag_ref in repo.tags)
    raw_df = pd.DataFrame(data=source, columns=["name", "timestamp"])
    sorted_df = raw_df.sort_values(by=["timestamp"])
    graph_df = sorted_df[sorted_df['timestamp'] > since].copy()
    graph_df['interval'] = graph_df['timestamp'].diff()
    return graph_df


print("Okay")

Okay


we will need to generate a dataframe for the tags, with label and timestamp

Let's filter this down to the given date and later

In [114]:
repo = git.Repo("../../ClassDojo/dojo-android")
start_date = datetime.today().astimezone() - timedelta(days=60)
graph_df = create_tag_interval_df(repo, start_date)
graph_df

,name,timestamp,interval
390,production/v6.37.0-codepush.v142,2024-06-10 16:11:02-07:00,NaT
377,production/6.38.0,2024-06-14 18:55:38+00:00,3 days 19:44:36
389,production/v6.36.0-codepush.v143,2024-06-14 16:09:18-07:00,0 days 04:13:40
391,production/v6.37.0-codepush.v144,2024-06-14 16:09:18-07:00,0 days 00:00:00
394,production/v6.38.0-codepush.v147,2024-06-14 16:09:18-07:00,0 days 00:00:00
392,production/v6.37.0-codepush.v145,2024-06-17 15:57:43-04:00,2 days 20:48:25
393,production/v6.38.0-codepush.v146,2024-06-17 15:57:43-04:00,0 days 00:00:00
865,v6.39.0,2024-06-21 14:01:35-04:00,3 days 22:03:52
866,v6.40.0,2024-06-28 15:35:31-04:00,7 days 01:33:56
867,v6.41.0,2024-07-04 10:15:36+02:00,5 days 12:40:05


This should give us something to work with

In [115]:
figure = px.bar(graph_df, x="timestamp", y="interval", title='Release Delay')
figure.show()


In [116]:
earlier_label = graph_df.iloc[0]['name']
diffcounts = []
for index, data in graph_df.iterrows():
    current_label = data['name']
    diffs = repo.commit(earlier_label).diff(current_label)
    diffcounts.append(len(diffs))
    earlier_label = current_label

diffcounts


[0,
 499,
 963,
 607,
 503,
 499,
 501,
 671,
 646,
 859,
 131,
 976,
 2,
 45,
 858,
 1386,
 1682,
 858,
 2003,
 30,
 156]

In [117]:
graph_df['timestamp'] = pd.to_datetime(graph_df['timestamp'], utc=True)

In [118]:
graph_df['diff_counts'] = diffcounts

In [119]:
graph_df

,name,timestamp,interval,diff_counts
390,production/v6.37.0-codepush.v142,2024-06-10 23:11:02+00:00,NaT,0
377,production/6.38.0,2024-06-14 18:55:38+00:00,3 days 19:44:36,499
389,production/v6.36.0-codepush.v143,2024-06-14 23:09:18+00:00,0 days 04:13:40,963
391,production/v6.37.0-codepush.v144,2024-06-14 23:09:18+00:00,0 days 00:00:00,607
394,production/v6.38.0-codepush.v147,2024-06-14 23:09:18+00:00,0 days 00:00:00,503
392,production/v6.37.0-codepush.v145,2024-06-17 19:57:43+00:00,2 days 20:48:25,499
393,production/v6.38.0-codepush.v146,2024-06-17 19:57:43+00:00,0 days 00:00:00,501
865,v6.39.0,2024-06-21 18:01:35+00:00,3 days 22:03:52,671
866,v6.40.0,2024-06-28 19:35:31+00:00,7 days 01:33:56,646
867,v6.41.0,2024-07-04 08:15:36+00:00,5 days 12:40:05,859


In [120]:
figure = px.scatter(
    graph_df.iloc[-25:],
    x="timestamp",
    y="diff_counts",
    size="diff_counts",
    color="diff_counts",
    title='Release Size and Frequency',
    hover_data=["timestamp", "name", "interval"]
)
figure.update_layout(autosize=True)
figure.update_traces(mode="markers+lines")
figure.show()

In [121]:
dir(figure)

['__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_annotation_like',
 '_allow_disable_validation',
 '_animation_duration_validator',
 '_animation_easing_validator',
 '_batch_layout_edits',
 '_batch_trace_edits',
 '_bracket_re',
 '_build_dispatch_plan',
 '_build_update_params_from_batch',
 '_config',
 '_data',
 '_data_defaults',
 '_data_objs',
 '_data_validator',
 '_dispatch_layout_change_callbacks',
 '_dispatch_trace_change_callbacks',
 '_filter_by_selector',
 '_frame_objs',
 '_frames_validator',
 '_get_child_prop_defaults',
 '_get_child_props',
 '_get_subplot_coordinates',
 '_get_subplot

In [122]:
hovertemplate = "%{name}"
figure = px.line(
    graph_df,
    x="timestamp",
    y="diff_counts",
    title='Release Frequency and Size',
    hover_name="name",
    markers=True
)
#figure.update_traces(mode="markers")
figure.update_layout(hovermode="x unified")
figure.show()

In [123]:
graph_df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 21 entries, 390 to 878
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype              
---  ------       --------------  -----              
 0   name         21 non-null     object             
 1   timestamp    21 non-null     datetime64[ns, UTC]
 2   interval     20 non-null     timedelta64[ns]    
 3   diff_counts  21 non-null     int64              
dtypes: datetime64[ns, UTC](1), int64(1), object(1), timedelta64[ns](1)
memory usage: 840.0+ bytes


Add a change-type stacked graph based on diffs.


In [124]:
by_date = lambda x: x.commit.authored_datetime
sorted_tags = sorted(repo.tags, key=by_date)

In [125]:

last_20 = sorted_tags[-20:]
for x in last_20:
    print(by_date(x), x)


2024-06-14 18:55:38+00:00 production/6.38.0
2024-06-14 16:09:18-07:00 production/v6.36.0-codepush.v143
2024-06-14 16:09:18-07:00 production/v6.37.0-codepush.v144
2024-06-14 16:09:18-07:00 production/v6.38.0-codepush.v147
2024-06-17 15:57:43-04:00 production/v6.37.0-codepush.v145
2024-06-17 15:57:43-04:00 production/v6.38.0-codepush.v146
2024-06-21 14:01:35-04:00 v6.39.0
2024-06-28 15:35:31-04:00 v6.40.0
2024-07-04 10:15:36+02:00 v6.41.0
2024-07-08 21:07:37+00:00 v6.42.0
2024-07-12 11:26:56+00:00 v6.43.0
2024-07-12 09:46:17-05:00 v6.43.0-codepush.v148
2024-07-12 18:21:18-04:00 v6.44.0
2024-07-19 15:25:11+00:00 v6.45.0
2024-07-26 10:42:25-07:00 v6.46.0
2024-07-31 15:50:09-07:00 v6.44.0-codepush.v150
2024-07-31 15:50:09-07:00 v6.45.0-codepush.v149
2024-08-02 15:33:20+02:00 v6.47.0
2024-08-02 18:26:19-04:00 v6.47.0-codepush.v151
2024-08-05 12:55:55+02:00 v6.48.0


In [126]:

start = last_20[0]
start


<git.TagReference "refs/tags/production/6.38.0">

In [127]:
from collections import Counter

change_name = {
    "A": "Files Added",
    "D": "Files Deleted",
    "R": "Files Renamed",
    "M": "Files Modified"
}


def change_series(start, last_20):
    for tag in last_20:
        diffs = start.commit.diff(tag.commit)
        yield {
            'Date': start.commit.committed_datetime.date(),
            'Name': start.name,
            **Counter(change_name[x.change_type] for x in diffs)
        }
        start = tag


change_df = pd.DataFrame(change_series(start, last_20))
change_df

,Date,Name,Files Modified,Files Renamed,Files Deleted,Files Added
0,2024-06-14,production/6.38.0,NaN,NaN,NaN,NaN
1,2024-06-14,production/6.38.0,683.0,59.0,103.0,118.0
2,2024-06-14,production/v6.36.0-codepush.v143,424.0,41.0,69.0,73.0
3,2024-06-14,production/v6.37.0-codepush.v144,407.0,20.0,47.0,29.0
4,2024-06-17,production/v6.38.0-codepush.v147,403.0,20.0,29.0,47.0
5,2024-06-17,production/v6.37.0-codepush.v145,404.0,20.0,48.0,29.0
6,2024-06-17,production/v6.38.0-codepush.v146,571.0,20.0,29.0,51.0
7,2024-06-21,v6.39.0,384.0,122.0,39.0,101.0
8,2024-06-28,v6.40.0,460.0,206.0,72.0,121.0
9,2024-07-04,v6.41.0,82.0,1.0,14.0,34.0


In [128]:
figure = px.bar(
    change_df,
    title=f"Changes Across Tags for {os.path.basename(os.path.dirname(repo.common_dir))}",
    x="Name",
    y=list(change_name.values()),
    labels=["Added", "Deleted", "Modified", "Removed"],
    hover_name="Name",
    hover_data=["Date"],
    text_auto='.2s'
)
figure.show()

In [129]:
figure = px.area(
    change_df,
    title=f"Changes Across Tags for {os.path.basename(os.path.dirname(repo.common_dir))}",
    x="Name",
    y=list(change_name.values()),
    labels=["Added", "Deleted", "Modified", "Removed"],
    hover_name="Name",
    hover_data=["Date"],
)
figure.show()